In [1]:
from torchvision import transforms
from models.classifiers import chexnet, chexnet_train_one_epoch, chexnet_test_net, fit_chexnet
import torch.optim
from torch.utils.data import DataLoader
from dotenv import load_dotenv
import os
from util.plot_tools import show_and_save_series_plot, save_series_plot
from datahandling.transforms import to_numeric_label, to_class_int
from dataset.chestxray import ChestXRayDataset
from torch.utils.tensorboard import SummaryWriter
from accelerate import Accelerator

# Load Environment variables

In [2]:
load_dotenv()
db_path = os.getenv("DB_PATH")
img_dir_name = os.getenv("IMG_DIR")

annotation_folder = os.path.join(db_path, "training_sets/chexnet")
img_dir = os.path.join(db_path, img_dir_name)
torch.cuda.empty_cache()

# Load Data

In [4]:
device = "cuda"
print(torch.cuda.get_device_name(device))
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

batch_size = 16
num_workers = 1

train_set = ChestXRayDataset(os.path.join(annotation_folder, "train_gh.csv"), img_dir, read_lib="pil", transform=transform, target_transform=to_numeric_label)
val_set = ChestXRayDataset(os.path.join(annotation_folder, "validation_gh.csv"), img_dir, read_lib="pil", transform=transform, target_transform=to_numeric_label)
test_set = ChestXRayDataset(os.path.join(annotation_folder, "test_gh.csv"), img_dir, read_lib="pil", transform=transform, target_transform=to_numeric_label)

train_loader = DataLoader(train_set, batch_size, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_set, batch_size, num_workers=num_workers, pin_memory=True)
test_set = DataLoader(test_set, batch_size, num_workers=num_workers, pin_memory=True)

Tesla V100-PCIE-16GB


In [5]:
model = chexnet(len(ChestXRayDataset.target_labels), device)
len(val_loader)

702

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-5)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=1)
# loss_fn = torch.nn.BCEWithLogitsLoss(size_average=True)
loss_fn = torch.nn.BCELoss(reduction="mean")
#loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
num_epochs = 100

avg_train_loss_series, avg_val_loss_series, avg_val_acc_series = fit_chexnet(
    model,
    optimizer, 
    lr_scheduler, 
    loss_fn, 
    train_loader, 
    val_loader, 
    num_epochs,
    len(ChestXRayDataset.target_labels),
    "result/chexnet/weights_experimental",
    device)

  0%|          | 0/100 [00:00<?, ?it/s]

Starting epoch 1 out of 100



 10%|█         | 496/4905 [06:41<53:53,  1.36it/s]

Last average training loss at batch 500 out of 4905: 1.5571964521706105



 19%|█▉        | 931/4905 [11:42<46:23,  1.43it/s]

...


 39%|███▉      | 1914/4905 [30:08<42:57,  1.16it/s]

Last average training loss at batch 2000 out of 4905: 1.5639693871885538



 50%|█████     | 2462/4905 [36:51<30:57,  1.31it/s]

Last average training loss at batch 2500 out of 4905: 1.703874213591218



 57%|█████▋    | 2790/4905 [40:11<23:47,  1.48it/s]

In [ ]:
result_folder = "result/chexnet"
save_series_plot(avg_train_loss_series, "training loss", result_folder, "chexnet_train_loss", True)
save_series_plot(avg_val_loss_series, "validation loss", result_folder, "chexnet_val_loss", True)
save_series_plot(avg_val_acc_series, "validation accuracy", result_folder, "chexnet_val_acc", True)

In [ ]:
# Validation block